# Subset NextGen HydroFabric on S3 for New York City

**Authors:**  
   - [Javed Ali](https://www.javedali.net/) (University of Central Florida)  
 
    
**Date:** 06.26.2023   

**Description**:  

The purpose of this Jupyter Notebook is to prepare inputs required to execute the [NOAA Next Generation (NextGen) Water Resource Modeling Framework](https://github.com/NOAA-OWP/ngen). These inputs consist of the following components: 

- Hydrologic and hydrodynamic graphs based on the National Hydrologic Geospatial (Hydrofabric) data which includes catchments, nexus, and flowlines. 
- Model domain parameters represented as configuration files.
- Meteorological forcing data.

The Hydrofabric data can be accessed publicly through the AWS catalog. In this notebook, we use the **pre-release** version of the dataset, which represents the most recent version available on the Amazon S3 Bucket at the time of developing this notebook (https://nextgen-hydrofabric.s3.amazonaws.com/index.html#pre-release/). The configuration files encompass model default parameters, formulations, input and output paths, simulation time step, initial conditions, and other relevant settings. This example demonstrates the retrieval of hydrofabric data, followed by the extraction of necessary infromation for creating the parameter configuration file. These files are created for running Conceptual Functional Equivalent (CFE) model and Simple Logical Tautology Handler (SLoTH) in the NGEN framework. To prepare forcing data, run the *ngen-hydrofabric-subset.ipynb* Jupyter Notebook.

**Software Requirements**:  

The software and operating system versions used to develop this notebook are listed below. To avoid encountering issues related to version conflicts among Python packages, we recommend creating a new environment variable and installing the required packages specifically for this notebook.

Tested on: MacOS Ventura 13.2.1  

> boto3: 1.26.76  
  dask-core: 2023.4.0  
  fiona: 1.9.3  
  fsspec: 2023.4.0  
  geopandas: 0.12.2   
  ipyleaflet: 0.17.2  
  ipywidgets: 7.7.5   
  matplotlib: 3.7.1   
  netcdf4: 1.6.3   
  numpy: 1.24.2  
  pandas: 2.0.0  
  requests: 2.28.2  
  s3fs: 2023.4.0  
  scipy: 1.10.1  
  xarray: 2023.4.1
  
**Supplementary Code**

This notebook relies on the following external scripts:  
- `subset.py` - A script originally written by Nels Frazier to subset the NGen Hydrofabric

**Acknowledgement:**

> This notebook is adapted from the code developed by CAUHSI team Tony Castronova and Irene Garousi-Nejad. 
---

In [27]:
# dealing with time and dates
import time
import datetime

# data manipulation
import pandas
import subset

# geospatial analysis
import pyproj
import geopandas
import ipyleaflet
import geopandas as gpd
from pathlib import Path
from sidecar import Sidecar
from requests import Request
from ipywidgets import Layout

# progress bar
from tqdm.notebook import tqdm

# system
import os

# building realization
import cfe_realization as r
from pathlib import Path

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

%reload_ext watermark

In [28]:
%watermark

Last updated: 2023-06-26T15:17:59.146523+00:00

Python implementation: CPython
Python version       : 3.9.16
IPython version      : 8.14.0

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 5.10.162+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 14
Architecture: 64bit



## 1. Create a map and load the Hydrofabric VPU geometries

The following cell creates an interactive map that encompasses the Hydrofabric VPU (Vector Processing Units) geometries. These geometries have been prepared ahead of time and are stored in a HydroShare [resource](https://www.hydroshare.org/resource/35e8c6023c154b6298fcda280beda849/). HydroShare offers convinient access to [WMS (Web Map Service)](https://docs.geoserver.org/latest/en/user/services/wms/index.html) and [WFS (Web Feature Service)](https://docs.geoserver.org/latest/en/user/services/wfs/index.html) capabilities, allowing us to easily display these geometries on an interactive map. These services facilitate the visualization and exploration of the data in a user-friendly manner.

Create a dictionary containing the HydroShare resource ID's of the NextGen hydrofabric boundaries. The vector data for these geometries can be obtained from the following HydroShare resource:

https://www.hydroshare.org/resource/3fbeb99e896b4d5f814aa512e8b87af5/

In [3]:
geometry_urls = {
'all_regions':'35e8c6023c154b6298fcda280beda849',
'16':  'e8ddee6a8a90484fa7a976458e79c0c3',
'01':  '5f0e81c665314967a1e15e4ae672aaae',
'02':  '131a6d6cc6514b558f968716783d7d47',
'03N': '38c84132987243c2a49ffb9d178f3162',
'03S': '5d9cdd0b6851460aaccd0c83557e4a6c',
'03W': '5674050a194c41b8a61f000c94c27983',
'04':  'd161033e07634d6199ae136a24807f22',
'05':  '47113551c63b41daa53465aee6cb69e9',
'06':  '1302f07176cd46e2ab70db730e601682',
'07':  'b380393bebaf47e68afd98fb15f4ff10',
'08':  '2391aadf1f4440499e7b61b4dcc41d94',
'09':  '27670ef43fbf42be914e1fca7d41ce0b',
'10L': 'b5028b1c8b5240f8b7deb3bcebc2f005',
'10U': 'b6dca803df5a4a8c8120512ccdfe8ba9',
'11':  '8e7a4c951c8241269e47ee461c1d9ef3',
'12':  '8ea1c9e098f044318777bf283c1fc0ad',
'13':  'b166308dffed4db39083393a894c3694',
'15':  '68501dc3b6214aca8d92aaae75aee941',
'16':  '1244ac2f25b0442cacece320424c6756',
'17':  'da20b06af50d4adab080597ae4ae8c46',
'18':  'ca2e56965245476fbcb258b7d2aec7ab',
'14':  '2d78b60ad0cf469daced4c4aa37764ad',
}

In [4]:
defaultLayout=Layout(width='960px', height='940px')

map_center = (40.730610, -73.935242) # NYC, NY
m = ipyleaflet.Map(
    basemap=ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.OpenStreetMap.Mapnik, layout=defaultLayout),
    center=map_center,
    zoom=9,
    scroll_wheel_zoom=True,
    tap=False
    )

# add USGS Gages
m.add_layer(
    ipyleaflet.WMSLayer(
        url='http://arcgis.cuahsi.org/arcgis/services/NHD/usgs_gages/MapServer/WmsServer',
        layers='0',
        transparent=True,
        format='image/png',
        min_zoom=8,
        max_zoom=18,
        )
)

# add the CONUS VPU boundaries
hs_wms_res_all = f'HS-{geometry_urls["all_regions"]}'
m.add_layer(
    ipyleaflet.WMSLayer(
        url=f'https://geoserver.hydroshare.org/geoserver/{hs_wms_res_all}/wms?',
        layers='vpu_boundaries',
        format='image/png',
        transparent=True,
        opacity=0.5,
        min_zoom=4,
        max_zoom=8
    )
)

# add the watershed VPU boundaries for each region.
#for vpu in ['01', '02','03N','03S','03W', '04','05','06','07','08','09','10L','10U','11','12','13','14','15','16','17','18']:
for vpu in ['02']:
    hs_wms_res = f'HS-{geometry_urls[vpu]}'
    m.add_layer(
        ipyleaflet.WMSLayer(
            url=f'https://geoserver.hydroshare.org/geoserver/{hs_wms_res}/wms?',
            layers=f'{vpu}_boundaries',
            format='image/png',
            transparent=True,
            opacity=0.5,
            min_zoom=8,
            max_zoom=18
        )
    )


## 2. Enable interactive geometry selection 

Now, let's introduce an event handler that empowers us to highlight and store geometries that are clicked on the map. The following function utilizes the `WFS` endpoint to fetch the shape's boundary when a geometry is clicked on the map. This boundary will be drawn on the map, enabling us to visually identify the selected area.Additionally, we will store the information about the selected area for later use, particularly when we need to collect the upstream catchments associated with the selected area. 

In [5]:
selected_df = None

def handle_map_interaction(**kwargs):
    global selected_df
    
    if kwargs.get('type') == 'click':
        hs_wms_vpu_all = f'HS-{geometry_urls["all_regions"]}'
        coords = kwargs['coordinates'] 
        url = f'https://geoserver.hydroshare.org/geoserver/{hs_wms_vpu_all}/wfs?' \
               'service=wfs&version=2.0.0&' \
              f'request=getFeature&' \
               'srsName=EPSG:4269&' \
              f'bbox={coords[1]},{coords[0]},{coords[1]},{coords[0]},EPSG:4269&' \
              f'typeName=vpu_boundaries&' \
               'outputFormat=json&' \
               'PropertyName=VPU'
        print(url)

        q = Request('GET', url).prepare().url
        df = gpd.read_file(q, format='json')
        
        # exit if a VPU is not found, i.e. a user doesn't click on the layer
        if len(df) == 0: return
        
        VPU = df.VPU.values[0]
        hs_wms_res = f'HS-{geometry_urls[VPU]}'
        print(f'You selected VPU {VPU}')
        url = f'https://geoserver.hydroshare.org/geoserver/{hs_wms_res}/wfs?' \
               'service=wfs&version=2.0.0&' \
              f'request=getFeature&' \
               'srsName=EPSG:4269&' \
              f'bbox={coords[1]},{coords[0]},{coords[1]},{coords[0]},EPSG:4269&' \
              f'typeName={VPU}_boundaries&' \
               'outputFormat=json&'

        q = Request('GET', url).prepare().url
        df = gpd.read_file(q, format='json')

        # exit if a VPU is not found, i.e. a user doesn't click on the layer
        if len(df) == 0: return
    
        # save vpu region, convert crs, and save selection for later
        df['VPU'] = VPU 
        df = df.to_crs('EPSG:4269')
        selected_df = df
        
        if type(m.layers[-1]) == ipyleaflet.WKTLayer:
            m.remove_layer(m.layers[-1])
            
        # display the watershed boundary on the map
        m.add_layer(ipyleaflet.WKTLayer(wkt_string=df.iloc[0].geometry.wkt))
        
m.on_interaction(handle_map_interaction)

Display the map

In [6]:
sc = Sidecar(title='NGEN HydroFabric Map')
with sc:
    display(m)

Select the VPU of interest. Run the following code cell to print the `id` of the selected area. 

In [18]:
selected_df.id

0    wb-694722
Name: id, dtype: object

## 3. Subset hydrofabric data for the selected area

The following code passes the `id` and `VPU` values of the geometries selected on the map (`selected_df`) to the hydrofabric subsetting script (`subset.py`). The subsetting algorithm implemented in the code adopts a reverse tracing technique called `subset_upstream`. It systematically identifies and selects all the upstream divides, catchments, nexuses, and flowlines starting from the most downstream nexus linked to the chosen geometries.

In [19]:
ids_list = 'wb-694856', 'wb-694725', 'wb-694855', 'wb-694724', 'wb-694854', 'wb-694723', 'wb-698891', 'wb-694722'

ids = list(ids_list) # selected_df.id.values, you can also put id manually ['wb-2917533'] 
vpus = ['02'] # list(selected_df.VPU.values) #['16'] 

output_files = []
for i in range(0, len(ids)):    
    print(50*'-'+f'\nProcessing VPU {vpus}, {ids[i]} ')
    st = time.time()
    # build the hydrofabric_url
    # the complete dataset can be found at: https://nextgen-hydrofabric.s3.amazonaws.com/index.html#pre-release/
    hydrofabric_url = f's3://nextgen-hydrofabric/pre-release/nextgen_02.gpkg' #{vpus[i]} in place of 02
    subset.subset_upstream(hydrofabric_url, ids[i])
    
    # move these files into a subdir to keep things orderly
    counter = 1
    outpath = ids[i]
    while os.path.exists(outpath):
        outpath = ids[i] + "_" + str(counter)
        counter += 1
    os.mkdir(outpath)
    for subdir in ['config', 'forcings', 'outputs']:
        os.mkdir(os.path.join(outpath, subdir))

    for f in [f'{ids[i]}_upstream_subset.gpkg',
              'catchments.geojson',
              'crosswalk.json',
              'flowpath_edge_list.json',
              'flowpaths.geojson',
              'nexus.geojson',
              'cfe_noahowp_attributes.csv']:
        os.rename(f, os.path.join(outpath, 'config', f))
        
    # output_files.append(f'{ids[i]}_upstream_subset.gpkg')
    print(f'Output files located at: {outpath}')
    print(f'Completed in {time.time() - st} seconds\n'+50*'-')    

outdir = Path(outpath)

--------------------------------------------------
Processing VPU ['02'], wb-694856 
s3://nextgen-hydrofabric/pre-release/nextgen_02.gpkg
Building Graph Network


Output files located at: wb-694856
Completed in 120.55004477500916 seconds
--------------------------------------------------
--------------------------------------------------
Processing VPU ['02'], wb-694725 
s3://nextgen-hydrofabric/pre-release/nextgen_02.gpkg


Building Graph Network


Output files located at: wb-694725
Completed in 98.91242289543152 seconds
--------------------------------------------------
--------------------------------------------------
Processing VPU ['02'], wb-694855 
s3://nextgen-hydrofabric/pre-release/nextgen_02.gpkg


Building Graph Network


...

Output files located at: wb-694723
Completed in 95.0460114479065 seconds
--------------------------------------------------
--------------------------------------------------
Processing VPU ['02'], wb-698891 
s3://nextgen-hydrofabric/pre-release/nextgen_02.gpkg


Building Graph Network


Output files located at: wb-698891
Completed in 95.53010201454163 seconds
--------------------------------------------------
--------------------------------------------------
Processing VPU ['02'], wb-694722 
s3://nextgen-hydrofabric/pre-release/nextgen_02.gpkg


Building Graph Network


Output files located at: wb-694722
Completed in 97.15651607513428 seconds
--------------------------------------------------


## 4. Add the Subset Hydrofabric to the map


Execute the following code cell to add the subset catchments and rivers from the geopackage file  as visual overlays on the map. In this section, we begin by reading the shapefiles stored within the geopackage file. We then proceed to transform the projection system of these shapes into Web Mercator, the desired coordinate reference system for our map. Next, we create a WKTLayer for each catchment and river in the subset shapefiles. A WKTLayer enables us to represent the shape's geometry using a WKT string, which provides a concise description of its spatial properties.

In [20]:
os.getcwd()

'/home/jovyan/data/cuahsi-workshop/notebooks/ngen'

In [21]:
outdir

PosixPath('wb-694722')

In [22]:
ids_list = ['wb-694856', 'wb-694725', 'wb-694855', 'wb-694724', 'wb-694854', 'wb-694723', 'wb-698891', 'wb-694722']

# define the target projection as EPSG:4269 - Web Mercator
# this is the default crs for leaflet
target_crs = pyproj.Proj('4269')

# Dictionaries to store catchments and rivers data for each id
catchments_dict = {}
rivers_dict = {}

for i in ids_list:
    # outdir
    out_dir = Path(i)

    # read the shapes
    catchments = geopandas.read_file(f'{out_dir}/config/{i}_upstream_subset.gpkg',
                              layer='divides')
    rivers = geopandas.read_file(f'{out_dir}/config/{i}_upstream_subset.gpkg',
                              layer='flowpaths')

    # transform the shapefile into EPSG:4269
    catchments = catchments.to_crs(target_crs.crs)
    rivers = rivers.to_crs(target_crs.crs)

    # Store the catchments and rivers data for this id in the dictionaries
    catchments_dict[i] = catchments
    rivers_dict[i] = rivers

# Now iterate over the stored data and add to the map
for i in ids_list:
    # add the catchments to the map
    for idx, shape in catchments_dict[i].iterrows():
        wkt = ipyleaflet.WKTLayer(wkt_string=shape.geometry.wkt)
        wkt.style = {'color': 'green'}
        m.add_layer(wkt)

    # add the rivers to the map
    for idx, shape in rivers_dict[i].iterrows():
        wkt = ipyleaflet.WKTLayer(wkt_string=shape.geometry.wkt)
        wkt.style = {'color': 'blue'}
        m.add_layer(wkt)


In [40]:
# # read the shapes
# catchments = geopandas.read_file(f'{outdir}/config/{ids[0]}_upstream_subset.gpkg', # ids = selected_df.id[0]
#                           layer='divides')
# rivers = geopandas.read_file(f'{outdir}/config/{ids[0]}_upstream_subset.gpkg',
#                           layer='flowpaths')

# # define the target projection as EPSG:4269 - Web Mercator
# # this is the default crs for leaflet
# target_crs = pyproj.Proj('4269')

# # transform the shapefile into EPSG:4269
# catchments = catchments.to_crs(target_crs.crs)
# rivers = rivers.to_crs(target_crs.crs)

# # add the catchments to the map
# for idx, shape in catchments.iterrows():
#     wkt = ipyleaflet.WKTLayer(wkt_string=shape.geometry.wkt)
#     wkt.style = {'color': 'green'}
#     m.add_layer(wkt)
    
# # add the rivers to the map
# for idx, shape in rivers.iterrows():
#     wkt = ipyleaflet.WKTLayer(wkt_string=shape.geometry.wkt)
#     wkt.style = {'color': 'blue'}
#     m.add_layer(wkt)

## 5. Build Realization 

In [26]:
for directory in ids_list:
    # select directory
    out_dir = Path(directory)
    
    cfe_atts_path = out_dir.resolve()/'config/cfe_noahowp_attributes.csv'

In [29]:
time={'start_time': '2012-10-19 00:00:00', # 3-days before and after the hurricane sandy event
      'end_time'  : '2012-11-05 00:00:00',
      'output_interval': 3600,
      'nts': 2592,  # Number of timesteps (288 for 1 day)
     }

for directory in tqdm(ids_list):
    # select directory
    out_dir = Path(directory)
    
    r.create_cfe_realization(out_dir/'config',
                         cfe_atts_path,
                         time=time,
                         config_path=Path(out_dir.resolve()/'config'),
                         forcing_path=Path(out_dir.resolve()/'forcings'),
                         troute_path=Path(out_dir.resolve()/'config/ngen.yaml'),
                         binary_path=Path('/opt/shared')
                        )

  0%|          | 0/8 [00:00<?, ?it/s]

**Aditional step**: Activate and run the following code cell to modify realization files or NextGen In A Box usage.

In [ ]:
# %%bash

# wb_id=wb-2917533

# sed -i "s+/home/jovyan/data/notebooks/ngen/${wb_id}+/ngen/ngen/data+g" ./${wb_id}/config/realization.json
# sed -i "s+/home/jovyan/data/notebooks/ngen/${wb_id}+/ngen/ngen/data+g" ./${wb_id}/config/ngen.yaml